# Predict bad LS moves

This notebook develops simple models for predicting bad local search moves. Particularly, given nodes $U$ and $V$ in routes $R_U$ and $R_V$, it predicts whether each LS operator we currently have is likely to produce an improving solution if the operator were applied to these node pairs $U$ and $V$.

The motivation is that evaluating a full operator move is typically somewhat slow, whereas a fast and reasonably accurate prediction method can completely avoid such evaluations.

TODO:
- New data on all U/V combinations, not just neighbourhood restricted.
- Penalties? For feasibility?
- Learning rate and validation stuff?
- Better feature generation

In [1]:
%cd ..

D:\Projects\Python\Euro-NeurIPS-2022


In [2]:
%matplotlib inline

In [55]:
from collections import defaultdict
from contextlib import suppress
from dataclasses import dataclass
from enum import IntEnum
from glob import glob
import itertools
from pathlib import Path
import re

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold

import tools

In [16]:
DATA_PATH = Path("data/raw/")
INST_PATH = Path("instances/")

## Utilities

These can be used to quickly parse the raw results for a single instance into something that contains the same data, but in a more workable format.

In [27]:
@dataclass
class Route:
    clients: list[int]
    load: int
    tw: int

    def index(self, client: int) -> int:
        return self.clients.index(client)

    def __getitem__(self, idx: int) -> int:
        return self.clients[idx]
    
    def __len__(self) -> int:
        return len(self.clients)

@dataclass
class Record:
    op: int
    U: int
    V: int
    delta: int
    Ru: Route
    Rv: Route

def parse_file(file: str) -> list[Record]:
    def parse_record(record: list[str]) -> Record:
        op = int(record[0].strip())
        U, V, delta = map(int, record[1].strip().split(" "))
        _, *Ru = map(int, re.findall('[0-9]+', record[2].strip()))
        _, *Rv = map(int, re.findall('[0-9]+', record[3].strip()))
        Lu, Lv = map(int, record[4].split(" "))
        TWu, TWv = map(int, record[5].split(" "))

        return Record(op, U, V, delta, Route(Ru, Lu, TWu), Route(Rv, Lv, TWv))

    with open(file, 'r') as fh:
        args = [iter(fh)] * 6
        records = zip(*args)

        # This could have been a generator, but each file is only 100-ish MB
        # in size, so that comfortably fits in memory.
        return [parsed for record in records if (parsed := parse_record(record)).op not in [2, 6]]

## Data and feature generation

Operators (in the order of `main.cpp`):

0. $(1, 0)$-Exchange
1. $(2, 0)$-Exchange
2. $(2, 0)$-Reverse-Exchange
3. $(2, 2)$-Exchange
4. $(2, 1)$-Exchange
5. $(1, 1)$-Exchange
6. 2-OPT

Note that we currently ignore 2 (reverse exchange) and 6 (two-opt), and focus only on the $(N, M)$-Exchange operators.

In [18]:
op2nm = [
    (1, 0),
    (2, 0),
    None,
    (2, 2),
    (2, 1),
    (1, 1),
    None
]

In [19]:
class Features(IntEnum):
    DIST_PU_V = 0
    DIST_PV_U = 1
    DIST_VM_UN1 = 2
    DIST_UN_VM1 = 3
    DIST_PU_U = 4
    DIST_PV_V = 5
    DIST_UN_UN1 = 6
    DIST_VM_VM1 = 7
    TW_U_FEAS = 8
    TW_V_FEAS = 9
    LOAD_U_FEAS = 10
    LOAD_V_FEAS = 11

In [20]:
def make_features(instance: dict, records: list[Record]) -> np.array:
    dist = instance['duration_matrix']
    dist_max = dist.max()

    data = np.empty((len(records), len(Features)))

    for idx, record in enumerate(records):
        n, m = op2nm[record.op]

        idx_u = record.Ru.index(record.U) if record.U != 0 else -1
        idx_v = record.Rv.index(record.V) if record.V != 0 else -1

        pu = 0 if idx_u <= 0 else record.Ru[idx_u - 1]
        pv = 0 if idx_v <= 0 else record.Rv[idx_v - 1]

        dist_un_vm1 = 0
        dist_un_un1 = 0
        with suppress(IndexError):
            un = record.Ru[idx_u + n]
            un1 = record.Rv[idx_u + n + 1] if idx_u + n + 1 < len(record.Ru) else 0
            dist_un_un1 = dist[un, un1]

            vm1 = record.Rv[idx_v + m + 1]   
            dist_un_vm1 = dist[un, vm1]

        dist_vm_un1 = 0
        dist_vm_vm1 = 0
        with suppress(IndexError):
            vm = record.Rv[idx_v + m]
            vm1 = record.Rv[idx_v + m + 1] if idx_v + m + 1 < len(record.Rv) else 0
            dist_vm_vm1 = dist[vm, vm1]

            un1 = record.Ru[idx_u + n + 1]
            dist_vm_un1 = dist[vm, un1]

        # Proposed distances
        data[idx, Features.DIST_PU_V] = dist[pu, record.V] / dist_max if m > 0 else 0
        data[idx, Features.DIST_PV_U] = dist[pv, record.U] / dist_max
        data[idx, Features.DIST_VM_UN1] = dist_vm_un1 / dist_max if m > 0 else 0
        data[idx, Features.DIST_UN_VM1] = dist_un_vm1 / dist_max

        # Existing distances
        data[idx, Features.DIST_PU_U] = dist[pu, record.U] / dist_max
        data[idx, Features.DIST_PV_V] = dist[pv, record.V] / dist_max
        data[idx, Features.DIST_UN_UN1] = dist_un_un1 / dist_max
        data[idx, Features.DIST_VM_VM1] = dist_vm_vm1 / dist_max

        # Time window feasibility indicators (no time warp indicates feasibility)
        data[idx, Features.TW_U_FEAS] = record.Ru.tw == 0
        data[idx, Features.TW_V_FEAS] = record.Rv.tw == 0

        # Load feasibility (less than vehicle capacity is feasible)
        data[idx, Features.LOAD_U_FEAS] = record.Ru.load <= instance['capacity']
        data[idx, Features.LOAD_V_FEAS] = record.Rv.load <= instance['capacity']

    return data

In [21]:
def make_or_retrieve_data(file_loc: str) -> tuple[np.array, np.array]:
    cache_loc = DATA_PATH / (Path(file_loc).stem + '.npz')

    if cache_loc.exists():
        file = np.load(cache_loc)
        return file['X'], file['y']                
    
    instance = tools.read_vrplib(INST_PATH / file_loc)
    records = parse_file(DATA_PATH / file_loc)

    y = np.array([int(record.delta < 0) for record in records])
    X = make_features(instance, records)

    np.savez(cache_loc, X=X, y=y)
    return X, y

## Training and evaluation

In [78]:
def do_kfold(n_splits: int, weights: dict, files: list[Path]) -> list:
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=4)
    fold_results = []

    for idx, (train, test) in enumerate(kf.split(files), 1):
        print(f"Fold {idx}")
        model = SGDClassifier(loss="log_loss", class_weight=weights, random_state=0)

        for idx in train:
            X, y = make_or_retrieve_data(files[idx])
            model.partial_fit(X, y, [0, 1])

        scores = []
        for idx in test:
            X, y = make_or_retrieve_data(files[idx])
            precision, recall, *_ = score(y, model.predict(X), average='weighted')

            # Precision: number of relevant documents retrieved by a search divided by the total number of documents retrieved
            # Recall: number of relevant documents retrieved by a search divided by the total number of existing relevant documents
            scores.append([precision, recall])

        fold_results.append([np.mean(scores, axis=0), model.coef_[0], model.intercept_])

    return fold_results

The number of improvements appears to be roughly ~0.25% to ~0.3% of the total number of evaluated moves, so we give those a weight of $\frac{1}{0.003}$ to compensate.

In [79]:
weights = {0: 1, 1: 1 / 0.003}
files = sorted([Path(file.name) for file in DATA_PATH.glob("ORTEC-*.txt")])

In [84]:
vals = do_kfold(10, weights, files)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10


In [87]:
for (p, r), coefs, intercept in vals:
    print(f"{p:.3f} \t {r:.3f} \t {[round(coef, 2) for coef in coefs]} \t {intercept[0]:.2f}")

0.996 	 0.878 	 [-8.41, -7.77, -0.01, 0.07, 6.11, 8.91, 0.07, 4.27, -1.24, -0.58, -1.34, -0.52] 	 1.79
0.996 	 0.854 	 [-8.41, -7.82, -0.01, 0.02, 6.11, 8.95, 0.1, 4.33, -1.24, -0.58, -1.37, -0.53] 	 1.82
0.996 	 0.863 	 [-8.34, -7.74, 0.04, 0.0, 6.1, 8.91, 0.08, 4.27, -1.24, -0.58, -1.32, -0.52] 	 1.76
0.996 	 0.825 	 [-8.27, -7.65, -0.05, 0.02, 6.1, 8.86, 0.1, 4.38, -1.33, -0.85, -1.19, -0.4] 	 1.94
0.996 	 0.866 	 [-8.5, -7.82, -0.02, 0.01, 6.26, 9.04, 0.12, 4.39, -1.23, -0.58, -1.33, -0.48] 	 1.70
0.996 	 0.881 	 [-8.37, -7.77, -0.02, 0.06, 6.12, 8.9, 0.08, 4.28, -1.24, -0.58, -1.31, -0.51] 	 1.74
0.996 	 0.858 	 [-8.4, -7.85, -0.01, 0.02, 6.14, 8.98, 0.09, 4.33, -1.24, -0.58, -1.34, -0.5] 	 1.75
0.996 	 0.842 	 [-8.4, -7.77, 0.01, 0.03, 6.18, 8.93, 0.08, 4.3, -1.24, -0.58, -1.36, -0.5] 	 1.77
0.996 	 0.871 	 [-8.36, -7.62, -0.08, 0.05, 6.01, 8.86, 0.15, 4.23, -1.24, -0.57, -1.37, -0.51] 	 1.78
0.996 	 0.858 	 [-8.39, -7.79, 0.02, 0.07, 6.06, 8.73, -0.07, 4.1, -1.26, -0.61, -1.33, 

In [92]:
_, coefs, intercept = max(vals, key=lambda val: val[0][1])  # max recall, since precision is rather static across instances
print([round(coef, 2) for coef in coefs])
print(round(intercept[0], 2))

[-8.37, -7.77, -0.02, 0.06, 6.12, 8.9, 0.08, 4.28, -1.24, -0.58, -1.31, -0.51]
1.74
